# Pandas - Merging, grouping, aggregation, relationships

In [ ]:
import pandas as pd
import numpy as np
from IPython.display import display, IFrame

np.random.seed(42)

In [ ]:
from solutions import run_solution, show_solution

In [ ]:
# Read the data
imdb_titles = pd.read_parquet("../data/imdb_movie_titles.parquet")
imdb_ratings = pd.read_parquet("../data/imdb_movie_ratings.parquet")
tmdb = pd.read_parquet("../data/tmdb.parquet")

## Merging data

We would like to attach ratings to the movie titles dataset. 
Let's merge (join) the two IMDB data sets. That's quite simple because they share the `tconst` columns.

In [ ]:
imdb_ratings.sample(10)

Luckily, the datasets have a common column (tconst) and can be merged easily using the [`pandas.merge`](https://pandas.pydata.org/docs/reference/api/pandas.merge.html) function. We also specify a very useful `validate` parameter. In our case, `validate=“1:1”` checks if merge keys are unique in both left and right datasets.

The join type (`how` parameter in `pd.merge`) is *inner*. There are four join (merge) types:

![join types](joins.png "Join Types")

In [ ]:
imdb_rated = pd.merge(imdb_titles, imdb_ratings, on='tconst', how='inner', validate="1:1")

In [ ]:
imdb_rated.head(10)

In this case, we could also use the `tconst` column as index and either use `pd.merge(imdb_titles, imdb_ratings, left_index=True, right_index=True)` or the simplified `join` method:

In [ ]:
# we set the index to the tconst column here for demonstration
imdb_titles.set_index("tconst").join(imdb_ratings.set_index("tconst")).head(10)

Similarly, we can merge the third TMDB dataset. It also includes `tconst` but as `imdb_id` column. We will also merge only certain columns from TMDB.

In [ ]:
movies_rated = pd.merge(
    imdb_rated,
    tmdb[["imdb_id", "budget", "popularity", "revenue", "vote_average", "vote_count"]],
    left_on="tconst",
    right_on="imdb_id",
    validate="1:1",
)

In [ ]:
movies_rated.sample(5)

**Exercise drop_imdb_id**: You may have noticed that we have two duplicate columns: `tconst` and `imdb_id`. Drop the `imdb_id` column as an exercise.

In [ ]:
# exercise drop_imdb_id

# run_solution("drop_imdb_id")
# show_solution("drop_imdb_id")

### Consumer Price Index

We will use one more dataset for this workshop, not strictly related to movies: The US Consumer Price Index, available from https://data.bls.gov/timeseries/CUUR0000SA0. This time, we read an unprocessed Excel data file. The format is not ideal but the `read_excel` function is flexible enough to handle this file.

In [ ]:
cpi = pd.read_excel("../data/cpi.xlsx", skiprows=10, header=1).convert_dtypes().rename(columns={"Year": "year"})
cpi

**Exercise cpi_2022**: Calculate and insert `Annual` value for 2022 based on the mean of available months (do not consider the number of days per month).

In [ ]:
# exercise cpi_2022

# run_solution("cpi_2022")
# show_solution("cpi_2022")

You probably realised that merging this dataset will be a bit different as we do not have CPI related to individual movies. However, movies are created in certain years. We can thus associate CPI by the movie release year. Of course we introduce some (maybe non-trivial) inconsistency when we do some analyses when we recalculate the dollar value based on CPI because the values (e.g. budgets) are potentially related to other year(s).

Let's do the merge anyways and add a CPI value for each movie based on the release year average CPI:

In [ ]:
movies_rated_cpi_all = movies_rated.merge(
    cpi[["year", "Annual"]].rename(columns={"Annual": "CPI"}),
    on="year",
)

In [ ]:
movies_rated_cpi_all[["primaryTitle", "year", "CPI"]].sample(10)

**Exercise cpi_validate_merge:** Add the right `validate` parameter to the merge with CPI.

In [ ]:
# exercise cpi_validate_merge

# run_solution("cpi_validate_merge")
# show_solution("cpi_validate_merge")

## Visual analysis

Pandas provides covenience methods for plotting using [Matplotlib](https://matplotlib.org). We will not show them here (you can follow the official [10 min tutorial on plotting](https://pandas.pydata.org/docs/user_guide/10min.html#plotting)); instead, we make use of [Plotly](https://plotly.com/python/), a "graphing library makes interactive, publication-quality graph". In particular, we will use the [Plotly Express](https://plotly.github.io/plotly.py-docs/plotly.express.html#px) high level interface.

There are more libraries you can explore when you have more time, such as https://seaborn.pydata.org, https://bokeh.pydata.org/en/latest/, https://altair-viz.github.io/, http://holoviews.org. You can find an overview at https://pyviz.org/high-level/index.html.

In [ ]:
import plotly.express as px

Let's try to visually "answer" whether the number of votes is related to rating using a simple scatter plot:

In [ ]:
px.scatter(movies_rated_cpi_all, x="numVotes", y="averageRating")

This is already quite useful. With a minimalistic function call, we have an *interactive* plot right *in the notebook*. Let's play with it for a while:
* Zoom in / out.
* Hover over data points.

We can tune the plot a bit more. For example, the number of votes would be better on a logarithmic scale. We can also add some opacity to get a sense of the density of the data points.

In [ ]:
px.scatter(movies_rated_cpi_all, x="numVotes", y="averageRating", log_x=True, opacity=0.1)

A very important aspect that can be observed is that low number of votes introduce more noise than information. Let's remove those movies for the further analysis. 

In [ ]:
movies_rated_cpi = movies_rated_cpi_all.loc[
    (movies_rated_cpi_all["numVotes"] > 100) & (movies_rated_cpi_all["vote_count"] > 100)
]

In [ ]:
print(f"We have removed {(1 - movies_rated_cpi.size / movies_rated_cpi_all.size) * 100: .1f} % movies")

To get quick insights into relationships between multiple variables, we can use `scatter_matrix`. 
* It is better to select only certain columns using the `dimensions` parameter.
* `hover_name` lets you modify the hover title.
* `hover_data` adds columns to the hover box.

In [ ]:
px.scatter_matrix(
    movies_rated_cpi, 
    dimensions=["averageRating", "budget", "popularity", "revenue", "vote_average"],
    hover_name="primaryTitle",
    hover_data=["year", "averageRating", "budget", "popularity", "revenue", "vote_average"]
)

**Exercise hover_data**: add all columns to hover_data and reduce opacity in the previous plot

In [ ]:
# exercise hover_data

# run_solution("hover_data")
# show_solution("hover_data")

We can also quantify the relationship using (linear) correlation coefficients:

In [ ]:
movies_rated_cpi[["averageRating", "vote_average", "budget", "revenue", "popularity"]].corr()

**Exercise inflation_adjust**: Create inflation-adjusted columns `revenue_2022` and `budget_2022`. The adjusted values are calculated as:

$$
  value(2022) = \frac{value(year)}{CPI(year)} CPI(2022)
$$

In [ ]:
# exercise inflation_adjust

# run_solution("inflation_adjust")
# show_solution("inflation_adjust")


Let's explore the top 10 most expensive movies using nominal and adjusted budgets and see if there are any differences.

In [ ]:
movies_rated_cpi.sort_values("budget", ascending=False).head(10)[["primaryTitle", "year", "budget"]]

In [ ]:
# uncomment when exercise inflation_adjust is done
# movies_rated_cpi_2022.sort_values("budget_2022", ascending=False).head(10)[["primaryTitle", "year", "budget", "budget_2022"]]

Do more expensive movies have better ratings?

In [ ]:
movies_rated_cpi[["averageRating", "vote_average", "budget", "revenue"]].corr()

# it is better to use the inflation adjusted values
# movies_rated_cpi_2022[["averageRating", "vote_average", "budget_2022", "revenue_2022"]].corr()

**Exercise plot_budget_corr**: Create a scatter plot (or scatter plot matrix) to visualize some of the (non) correlations

In [ ]:
# exercise plot_budget_corr 

# run_solution("plot_budget_corr")
# show_solution("plot_budget_corr")


### Working with categorical data

The plots above are for numerical data (real or integer numbers in our case). What about working with some categories? We have some categories already available in the dataset, we can also create some artificially.

First, let's create a `decade` category, which corresponds to the decade of the release year.

In [ ]:
def interval_to_decade_name(interval):
    return str(interval.left)+"s"

decades = pd.cut(
    movies_rated_cpi["year"],
    bins=range(1890, 2031, 10),
).apply(interval_to_decade_name)

decades.tail(5)

We can now create [box plots](https://en.wikipedia.org/wiki/Box_plot) of average rating per decade. This will give us some sense about the "average quality of movies in each decade" (if we trust IMDB rating as a reasonable quality proxy) and also about some statistical properties of the ratings (quantiles, max / min values, skewness, outliers).

In [ ]:
px.box(
    movies_rated_cpi,
    x=decades,
    y="averageRating",
)

Have you relaized that the decades are not ordered? And the x-axis is labeled just as "x". This is because we made a shortcut and used `decades` data as `x` argument rather than merging the data first and use `x="decade"` 

**Exercise sort_decades**: 
1. Use either `assign` or `join` to add `"decade"` column to `movies_rated_cpi`. 
2. Either sort the values appropriately for the plotting (the categories appear in the plot in the same order as in the data) or use an explicit `category_orders` argument.

In [ ]:
# exercise sort_decades

# run_solution("sort_decades")
# show_solution("sort_decades")


To work with genres, we need to do some extra manipulation. We first need to split the `genres` strings as they are comma-separated values. We then use `explode` to create individual rows per every genre. This means that there will be possibly multiple rows per one movie.

In [ ]:
decades_and_genres = (
    movies_rated_cpi.assign(
        decade = pd.cut(
            movies_rated_cpi["year"],
            bins=range(1890, 2031, 10)
        ).apply(interval_to_decade_name),
        genres = movies_rated_cpi["genres"].str.split(",")
    )
    .rename({"genres": "genre"}, axis="columns")
    .explode("genre")
)

decades_and_genres

Let's see what happened. One (side) effect that the index is duplicated. We will use this to filter out movies with multiple genres.

In [ ]:
decades_and_genres.loc[decades_and_genres.index.duplicated(keep=False), ["primaryTitle", "genre"]].head(5)

It is better to reset this index to remove the duplicates.

In [ ]:
decades_and_genres = decades_and_genres.reset_index()

We can get a similar (more detailed) insight into the rating distribution per decade using histograms. We make use of the `facet_col` parameter.

In [ ]:
px.histogram(
    decades_and_genres.astype({"decade": str}),
    x="averageRating",
    facet_col="decade",
    facet_col_wrap=4,
    width=1000, 
    height=1000,
    histnorm="probability density",
    nbins=30,
)

**Exercise star_rating**: Create one more categorical variable called `star_rating`. Use [`pd.qcut`] (https://pandas.pydata.org/docs/reference/api/pandas.qcut.html) to assign 1 - 5 stars based on the *quntile* of the `averageRating` column. Check the value counts of this new column.

In [ ]:
# exercise star_rating

# run_solution("star_rating")
# show_solution("star_rating")


We can use more plot properties to work with more categories. For example, we can use `color` to distinguish genres while we still have decade on the x axis.

In [ ]:
px.violin(
    decades_and_genres.loc[decades_and_genres["genre"].isin(["Documentary", "Horror"])].sort_values("decade"),
    x="decade",
    y="averageRating",
    color="genre",
    violinmode="overlay",
)

## Grouping & aggregation

A common pattern in data analysis is grouping (or binning) data based on some property and getting some aggredate statistics.

*Example:* Group this workshop participants by nationality a get the cardinality (the size) of each group.

In [ ]:
grouped_by_genre = decades_and_genres.groupby('genre')

What did we get? 

In [ ]:
grouped_by_genre

What's this `DataFrameGroupBy` object? [Its use case is](http://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html):
* Splitting the data into groups based on some criteria.
* Applying a function to each group independently.
* Combining the results into a data structure.


Let's try a simple aggregate: mean of rating per type group:

In [ ]:
grouped_by_genre["averageRating"].mean().sort_values()

`GroupBy` objects have `agg` method which is quite versatile to describe what aggregations we need. As an example:

In [ ]:
grouped_by_genre.agg({"averageRating": ["mean", "median", "std"], "year": ["min", "max", "median"]})

## Pivoting

> pivot (third-person singular simple present pivots, present participle pivoting, simple past and past participle pivoted)
 **To turn on an exact spot.**
 
> A pivot table is a table of statistics that summarizes the data of a more extensive table ...
> Although pivot table is a generic term, Microsoft Corporation trademarked PivotTable in the United States in 1994.

Our pivoting task: Get a table with numbers of titles per year (as row) and type (as column).

One approach is to use `groupby`, `count` aggregation and `unstack`.

In [ ]:
grouped_by_year_and_type = decades_and_genres.groupby(['year', 'genre'])

In [ ]:
pivoted = (
    grouped_by_year_and_type["numVotes"]
    .sum()
    .unstack()
)
pivoted.tail()

There's a shortcut though, see if you we can use it.

**Exercise pivot_table:** Create the `pivoted` table using `pivot_table`:

In [ ]:
# exercise pivot_table

# run_solution("pivot_table")
# show_solution("pivot_table")


We can now use this to plot a kind of a histogram with colour for genres.

In [ ]:
px.imshow(pivoted)

Plotly actually has `density_heatmap`, which can do the aggregation (pivoting) and plotting at once.

In [ ]:
px.density_heatmap(decades_and_genres, x="genre", y="year", histfunc="sum")

A similar, more versatile function is [crosstab](https://pandas.pydata.org/docs/reference/api/pandas.crosstab.html).

In [ ]:
decades_vs_genres_crosstab = pd.crosstab(
    index=decades_and_genres["decade"],
    columns=decades_and_genres["genre"],
    values=decades_and_genres["numVotes"],
    aggfunc="sum"
)

decades_vs_genres_crosstab[["Documentary", "Horror"]]

**Exercise genre_normalize**: Normalize `decades_vs_genres_crosstab` per genre so the average over each genre is 1. Use px.imshow to plot the normalized data.

In [ ]:
# exercise genre_normalize

# run_solution("genre_normalize")
# show_solution("genre_normalize")


## Final mini-project - creative, unbounded, free-style

Here are some ideas of what you can do with the data.

* Do some analyses based on the 5-star rating.
  * E.g., visualize the distrubution of incomes in each 5-star rating group.
* More Group by decade / rating or language analyses.
* Create production_company (categorical) column from `tmdb["production_companies"]`, e.g. using `pd.json_normalize`. Bear in mind more companies can produce a single movie.
* Compare simple (arithmetic) mean `averageRating` in each group with `averageRating` average weighted by `numVotes` ($ \frac{\sum \rm{averageRating} \times \rm{numVotes}} {\sum \rm{numVotes}} $). Use `apply` and the `wavg` function from https://pbpython.com/weighted-average.html. This function is quite time and memory consuming and thus not ideal for large data sets. You can try to implement weighted average using standard `mean`. Check the performance with the `%timeit` magic.
* Use the 5-star rating for `hue` in an interesting plot (see e.g. https://seaborn.pydata.org/tutorial/relational.html for some inspiration)
 
A couple more ideas can be found in https://github.com/brandon-rhodes/pycon-pandas-tutorial or https://vvdwivedi.github.io/tmdb-data-analysis/.

After you have solved all of those, come up with your own quests - we may still be around and help you :-D

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=234de414-c5f7-4e4d-a314-25100ac19112' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>